In [15]:
!pip install elasticsearch-dsl
!pip install cassandra-driver
!pip install elasticsearch
!pip install confluent-kafka

In [48]:
from confluent_kafka import Consumer
from elasticsearch import Elasticsearch
from cassandra.cqlengine import connection
from cassandra.cluster import Cluster
from cassandra.policies import TokenAwarePolicy, RoundRobinPolicy
from elasticsearch_dsl import Search
from elasticsearch_dsl import A

In [17]:
# Apache Cassandra connection
list_of_ip = (['192.168.56.101', '192.168.56.102', '192.168.56.103'])
cluster = Cluster(list_of_ip,load_balancing_policy=TokenAwarePolicy(RoundRobinPolicy()))
session = cluster.connect()
session.set_keyspace('thirdeye_test')
connection.set_session(session)

C:\Users\harii\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Legacy execution parameters will be removed in 4.0. Consider using execution profiles.
  This is separate from the ipykernel package so we can avoid doing imports until


CQLEngineException: Failed to initialize: row_factory must be 'dict_factory'

In [18]:
# Elasticsearch connection
es = Elasticsearch()

## 1. Simple

### 1.1 What is the min/max/average delays for an airline on a given day/month/year? 

In [55]:
s = Search(using=es, index="my-index") \
    .query("match", Year=2017) \
    .query("match", Month=1) \
    .query("match", DayofMonth=1)

a = A('terms', field='category')
a.metric('avg_delay', 'avg', field='ArrDelayMinutes')\
    .bucket('max_delay', 'max', field='ArrDelayMinutes')
s.aggs.bucket('category_terms', a)

print(s.to_dict())
response = s.execute()
for hit in response:
    print(hit)

{'query': {'bool': {'must': [{'match': {'Year': 2017}}, {'match': {'Month': 1}}, {'match': {'DayofMonth': 1}}]}}, 'aggs': {'category_terms': {'terms': {'field': 'category'}, 'aggs': {'avg_delay': {'avg': {'field': 'ArrDelayMinutes'}}, 'max_delay': {'max': {'field': 'ArrDelayMinutes'}}}}}}
<Hit(my-index/MTRE0XABWCivyfFAf-fq): {'Year': 2017, 'Quarter': 1, 'Month': 1, 'DayofMonth': 1, 'D...}>
<Hit(my-index/NjRE0XABWCivyfFAgOcJ): {'Year': 2017, 'Quarter': 1, 'Month': 1, 'DayofMonth': 1, 'D...}>
<Hit(my-index/czRE0XABWCivyfFAged0): {'Year': 2017, 'Quarter': 1, 'Month': 1, 'DayofMonth': 1, 'D...}>
<Hit(my-index/ijRE0XABWCivyfFAgef-): {'Year': 2017, 'Quarter': 1, 'Month': 1, 'DayofMonth': 1, 'D...}>
<Hit(my-index/mjRE0XABWCivyfFAgudd): {'Year': 2017, 'Quarter': 1, 'Month': 1, 'DayofMonth': 1, 'D...}>
<Hit(my-index/tDRE0XABWCivyfFAguf1): {'Year': 2017, 'Quarter': 1, 'Month': 1, 'DayofMonth': 1, 'D...}>
<Hit(my-index/tjRE0XABWCivyfFAg-cB): {'Year': 2017, 'Quarter': 1, 'Month': 1, 'DayofMonth': 

### 1.2 Top 3 Airlines that did not report the aircraft tail number (Aircraft tail number was null)

### 1.3 Were there any specific airport with maximum delays on a given day?

### 1.4 What is the min/max and average time between delays by delay type on a given day?

### 1.5 Categories the number of delays by delay type

### 1.6 What is the min, max and average time between the planned and actual arrival time of the aircraft by airline?

## 2. Moderate

### 2.1 What delay type is most common at each airport (over a month, quarter or year)?

### 2.2 Did privately managed airlines perform better than publicly traded ones?

### 2.3 What airline had the most delays each month?

## 3. Difficult

### 3.1 How did other airlines (peers) perform during bad weather (over a month, quarter or year)?

### 3.2. Is the airline at fault for the delay and should it compensate the customers?

### 3.3 Was there any correlation between delay trends and companies stock performance?

### 3.4 Did airlines with modernized fleet perform better during bad weather?